In [1]:
library(lme4)
library(sjPlot)
library(Hmisc)

library(gridExtra)
library(ggplot2)


# custom style for tab models
css = list(
    css.thead = 'border-top: double; text-align:center; font-style:normal; font-weight:bold; padding:0.1cm;',
    css.tdata = 'padding:0.05cm; padding-left:0.2cm; padding-right:0.2cm; text-align:left; vertical-align:top;',
    css.depvarhead = 'text-align:center; border-bottom:1px solid; font-style:italic; font-weight:bold',
    css.summary = 'padding-top:0.05cm; padding-bottom:0.05cm;'
  )

Loading required package: Matrix

Learn more about sjPlot with 'browseVignettes("sjPlot")'.

Loading required package: lattice

Loading required package: survival

Loading required package: Formula

Loading required package: ggplot2


Attaching package: ‘Hmisc’


The following objects are masked from ‘package:base’:

    format.pval, units




# Read Data

In [2]:
wtdata <- read.csv("data/dataset_A_linked.csv")

# Scale and center variables

In [3]:
# scale and center the election periods
wtdata$is_before_elections <- scale(wtdata$election_period_cat == 'A', scale=T)
wtdata$is_during_elections <- scale(wtdata$election_period_cat == 'B', scale=T)
wtdata$is_after_elections <- scale(wtdata$election_period_cat == 'C', scale=T)

# scaling the rest of the variables
wtdata$rank <- scale(wtdata$rank, scale=T)
# invert this value so that the odds ratio, so it the scaled scores
# match the intuition, i.e. the 1st result should have a higher (positive)
# value. The odds ratio are then easier to interpret.
wtdata$inv_rank <- -1 * wtdata$rank

# scale and center the representativeness
wtdata$news_representativeness <- scale(wtdata$news_representativeness, scale=T)

# scale and center engagement variables
wtdata$engagement_TD_B <- scale(wtdata$engagement_TD_B, scale=T)
wtdata$engagement_TD_C <- scale(wtdata$engagement_TD_C, scale=T)
wtdata$engagement_TD_D <- scale(wtdata$engagement_TD_D, scale=T)

# scale and center engagement variables
wtdata$visits_TD_B <- scale(wtdata$visits_TD_B, scale=T)
wtdata$visits_TD_C <- scale(wtdata$visits_TD_C, scale=T)
wtdata$visits_TD_D <- scale(wtdata$visits_TD_D, scale=T)

# scale and center engagement variables
wtdata$allnews_visits_TD_B <- scale(wtdata$allnews_visits_TD_B, scale=T)
wtdata$allnews_visits_TD_C <- scale(wtdata$allnews_visits_TD_C, scale=T)
wtdata$allnews_visits_TD_D <- scale(wtdata$allnews_visits_TD_D, scale=T)

# convert to factors the within subjects
wtdata$search_page <- as.factor(wtdata$search_page)
wtdata$participant <- as.factor(wtdata$participant)
wtdata$coredomain <- as.factor(wtdata$coredomain)

# Dataset B: engagement

In [4]:
# time dependent
td_model_B <- glmer(is_selected_result ~ 

                            # main factors
                              visits_TD_B
                            * inv_rank
                            * news_representativeness 
                    
                            # control by total news visits
                            + allnews_visits_TD_B

                            # control for demographics
                            + is_female
                            + is_west_germany
                            + age
                            + education
                            + political_alignment

                            # control for election day
                            + is_during_elections
                            + is_after_elections

                            # within subject controls
                            + (1|search_page)
                            + (1|coredomain)
                            + (1|participant)

                            , data=wtdata, family=binomial)


# save the model and display
htmlmodel <- tab_model(td_model_B, wrap.labels = F, CSS = css)
writeLines(text = htmlmodel$page.complete, con = "statistical_tests/absolute_visits/td_model_B.html")
IRdisplay::display_html( htmlmodel$page.content)


boundary (singular) fit: see help('isSingular')



is_selected_result 
 
 
 Predictors 
 Odds Ratios 
 CI 
 p 
 
 
 (Intercept) 
 0.23 
 0.21 – 0.25 
 <0.001 
 
 
 visits TD B 
 0.97 
 0.88 – 1.09 
 0.643 
 
 
 inv rank 
 3.09 
 2.82 – 3.37 
 <0.001 
 
 
 news representativeness 
 0.75 
 0.66 – 0.85 
 <0.001 
 
 
 allnews visits TD B 
 0.98 
 0.91 – 1.05 
 0.549 
 
 
 is female 
 0.99 
 0.92 – 1.06 
 0.745 
 
 
 is west germany 
 0.95 
 0.88 – 1.02 
 0.133 
 
 
 age 
 1.01 
 0.94 – 1.10 
 0.752 
 
 
 education 
 0.95 
 0.88 – 1.03 
 0.198 
 
 
 political alignment 
 1.04 
 0.97 – 1.12 
 0.266 
 
 
 is during elections 
 0.94 
 0.88 – 1.02 
 0.121 
 
 
 is after elections 
 1.01 
 0.94 – 1.08 
 0.872 
 
 
 visits TD B * inv rank 
 1.04 
 0.94 – 1.14 
 0.487 
 
 
 visits TD B * news representativeness 
 0.84 
 0.67 – 1.05 
 0.132 
 
 
 inv rank * news representativeness 
 1.14 
 1.00 – 1.31 
 0.052 
 
 
 (visits TD B * inv rank) * news representativeness 
 1.11 
 0.93 – 1.33 
 0.242 
 
 
 Random Effects 
 

 
 σ 2 
 3.29 
 

 
 τ 00 search_page 
 0.00 

 
 τ 00 participant 
 0.00 

 
 τ 00 coredomain 
 0.01 

 
 ICC 
 0.00 

 
 N search_page 
 1205 

 
 N coredomain 
 178 

 
 N participant 
 275 
 
 Observations 
 5323 
 
 
 Marginal R 2 / Conditional R 2 
 0.282 / 0.283

# Dataset C: Google associated engagement

In [5]:
# time independent
td_model_C <- glmer(is_selected_result ~ 

                            # main factors
                              visits_TD_C
                            * inv_rank
                            * news_representativeness 
                    
                            # control by total news visits
                            + allnews_visits_TD_C

                            # control for demographics
                            + is_female
                            + is_west_germany
                            + age
                            + education
                            + political_alignment

                            # control for election day
                            + is_during_elections
                            + is_after_elections

                            # within subject controls
                            + (1|search_page)
                            + (1|coredomain)
                            + (1|participant)

                            , data=wtdata, family=binomial)


# save the model and display
htmlmodel <- tab_model(td_model_C, wrap.labels = F, CSS = css)
writeLines(text = htmlmodel$page.complete, con = "statistical_tests/absolute_visits/td_model_C.html")
IRdisplay::display_html( htmlmodel$page.content)


boundary (singular) fit: see help('isSingular')



is_selected_result 
 
 
 Predictors 
 Odds Ratios 
 CI 
 p 
 
 
 (Intercept) 
 0.23 
 0.21 – 0.25 
 <0.001 
 
 
 visits TD C 
 0.99 
 0.90 – 1.08 
 0.834 
 
 
 inv rank 
 3.08 
 2.81 – 3.37 
 <0.001 
 
 
 news representativeness 
 0.75 
 0.66 – 0.86 
 <0.001 
 
 
 allnews visits TD C 
 0.94 
 0.87 – 1.01 
 0.097 
 
 
 is female 
 0.99 
 0.92 – 1.06 
 0.695 
 
 
 is west germany 
 0.94 
 0.88 – 1.01 
 0.117 
 
 
 age 
 1.02 
 0.94 – 1.10 
 0.673 
 
 
 education 
 0.95 
 0.88 – 1.03 
 0.194 
 
 
 political alignment 
 1.04 
 0.97 – 1.11 
 0.320 
 
 
 is during elections 
 0.94 
 0.87 – 1.01 
 0.095 
 
 
 is after elections 
 1.01 
 0.94 – 1.08 
 0.775 
 
 
 visits TD C * inv rank 
 0.98 
 0.89 – 1.09 
 0.739 
 
 
 visits TD C * news representativeness 
 0.95 
 0.82 – 1.09 
 0.445 
 
 
 inv rank * news representativeness 
 1.14 
 1.00 – 1.31 
 0.056 
 
 
 (visits TD C * inv rank) * news representativeness 
 1.09 
 0.92 – 1.29 
 0.330 
 
 
 Random Effects 
 

 
 σ 2 
 3.29 
 

 
 τ 00 search_page 
 0.00 

 
 τ 00 participant 
 0.00 

 
 τ 00 coredomain 
 0.01 

 
 N search_page 
 1205 

 
 N coredomain 
 178 

 
 N participant 
 275 
 
 Observations 
 5323 
 
 
 Marginal R 2 / Conditional R 2 
 0.283 / NA

# Dataset D: non Google associated engagement

In [6]:
# time independent
td_model_D <- glmer(is_selected_result ~ 

                            # main factors
                              visits_TD_D
                            * inv_rank
                            * news_representativeness 
                    
                            # control by total news visits
                            + allnews_visits_TD_D
                    
                            # control for demographics
                            + is_female
                            + is_west_germany
                            + age
                            + education
                            + political_alignment

                            # control for election day
                            + is_during_elections
                            + is_after_elections

                            # within subject controls
                            + (1|search_page)
                            + (1|coredomain)
                            + (1|participant)

                            , data=wtdata, family=binomial)

# save the model and display
htmlmodel <- tab_model(td_model_D, wrap.labels = F, CSS = css)
writeLines(text = htmlmodel$page.complete, con = "statistical_tests/absolute_visits/td_model_D.html")
IRdisplay::display_html( htmlmodel$page.content)

boundary (singular) fit: see help('isSingular')



is_selected_result 
 
 
 Predictors 
 Odds Ratios 
 CI 
 p 
 
 
 (Intercept) 
 0.23 
 0.21 – 0.25 
 <0.001 
 
 
 visits TD D 
 0.97 
 0.87 – 1.09 
 0.638 
 
 
 inv rank 
 3.09 
 2.82 – 3.37 
 <0.001 
 
 
 news representativeness 
 0.75 
 0.66 – 0.85 
 <0.001 
 
 
 allnews visits TD D 
 0.99 
 0.92 – 1.06 
 0.750 
 
 
 is female 
 0.99 
 0.92 – 1.06 
 0.772 
 
 
 is west germany 
 0.95 
 0.88 – 1.02 
 0.137 
 
 
 age 
 1.01 
 0.93 – 1.10 
 0.777 
 
 
 education 
 0.95 
 0.88 – 1.03 
 0.188 
 
 
 political alignment 
 1.04 
 0.97 – 1.12 
 0.256 
 
 
 is during elections 
 0.94 
 0.88 – 1.02 
 0.125 
 
 
 is after elections 
 1.00 
 0.94 – 1.08 
 0.893 
 
 
 visits TD D * inv rank 
 1.04 
 0.94 – 1.15 
 0.472 
 
 
 visits TD D * news representativeness 
 0.84 
 0.67 – 1.06 
 0.143 
 
 
 inv rank * news representativeness 
 1.14 
 1.00 – 1.31 
 0.051 
 
 
 (visits TD D * inv rank) * news representativeness 
 1.11 
 0.93 – 1.32 
 0.264 
 
 
 Random Effects 
 

 
 σ 2 
 3.29 
 

 
 τ 00 search_page 
 0.00 

 
 τ 00 participant 
 0.00 

 
 τ 00 coredomain 
 0.01 

 
 ICC 
 0.00 

 
 N search_page 
 1205 

 
 N coredomain 
 178 

 
 N participant 
 275 
 
 Observations 
 5323 
 
 
 Marginal R 2 / Conditional R 2 
 0.281 / 0.283